<a href="https://colab.research.google.com/github/fatemafaria142/Exploring-Large-Language-Models-in-Science-Exam-Question-Answering/blob/main/Llama_2_7b_chat_hf_and_LangChain_fine_tuning_SciQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets torch langchain -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━

#### **Dataset Link:** https://huggingface.co/datasets/sciq

In [3]:
from datasets import load_dataset

instruct_tune_dataset = load_dataset("sciq")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/11679 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [4]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 11679
    })
    validation: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 1000
    })
})

In [5]:
# Display information for 5 data points from the 'train' split
num_samples_to_show = 5
for i in range(num_samples_to_show):
    data = instruct_tune_dataset['train'][i]
    print(f"Data Point {i + 1}:")
    print("Question:", data['question'])
    print("Response:", data['correct_answer'])
    print("\n-----------------------------\n")

Data Point 1:
Question: What type of organism is commonly used in preparation of foods such as cheese and yogurt?
Response: mesophilic organisms

-----------------------------

Data Point 2:
Question: What phenomenon makes global winds blow northeast to southwest or the reverse in the northern hemisphere and northwest to southeast or the reverse in the southern hemisphere?
Response: coriolis effect

-----------------------------

Data Point 3:
Question: Changes from a less-ordered state to a more-ordered state (such as a liquid to a solid) are always what?
Response: exothermic

-----------------------------

Data Point 4:
Question: What is the least dangerous radioactive decay?
Response: alpha decay

-----------------------------

Data Point 5:
Question: Kilauea in hawaii is the world’s most continuously active volcano. very active volcanoes characteristically eject red-hot rocks and lava rather than this?
Response: smoke and ash

-----------------------------



### **We will use just a small subset of the data for this training example**

In [6]:
instruct_tune_dataset["train"] = instruct_tune_dataset["train"].select(range(3500))
instruct_tune_dataset["test"] = instruct_tune_dataset["test"].select(range(500))

#### **Creating a Mental Health Support Prompt Using LangChain**

In [14]:
from langchain.prompts import PromptTemplate
def create_prompt(sample):
    # Creating prompt for a large language model
    pre_prompt = """[INST] <<SYS>>\nPut yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges.\n"""
    prompt = pre_prompt + "Here is my question: {question}" +"[\INST]"+"\nExpert Response: {correct_answer}"

    # Extract context and response from the sample
    example_context = sample['question']
    example_response = sample['correct_answer']

    # Create a prompt using the template and the example
    prompt_template = PromptTemplate(template=prompt, input_variables=["question", "correct_answer"])
    single_turn_prompt = prompt_template.format(question=example_context, correct_answer=example_response)

    return single_turn_prompt


#### **Generating LangChain Prompt for a Training Subset Examples**

In [15]:
prompt = create_prompt(instruct_tune_dataset["train"][0])
print(prompt)

[INST] <<SYS>>
Put yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges.
Here is my question: What type of organism is commonly used in preparation of foods such as cheese and yogurt?[\INST]
Expert Response: mesophilic organisms


In [16]:
prompt = create_prompt(instruct_tune_dataset["train"][1])
print(prompt)

[INST] <<SYS>>
Put yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges.
Here is my question: What phenomenon makes global winds blow northeast to southwest or the reverse in the northern hemisphere and northwest to southeast or the reverse in the southern hemisphere?[\INST]
Expert Response: coriolis effect


In [17]:
prompt = create_prompt(instruct_tune_dataset["train"][2])
print(prompt)

[INST] <<SYS>>
Put yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges.
Here is my question: Changes from a less-ordered state to a more-ordered state (such as a liquid to a solid) are always what?[\INST]
Expert Response: exothermic


#### **Generating LangChain Prompt for a Testing Subset Examples**

In [18]:
prompt = create_prompt(instruct_tune_dataset["test"][0])
print(prompt)

[INST] <<SYS>>
Put yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges.
Here is my question: Compounds that are capable of accepting electrons, such as o 2 or f2, are called what?[\INST]
Expert Response: oxidants


In [19]:
prompt = create_prompt(instruct_tune_dataset["test"][1])
print(prompt)

[INST] <<SYS>>
Put yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges.
Here is my question: What term in biotechnology means a genetically exact copy of an organism?[\INST]
Expert Response: clone


In [20]:
prompt = create_prompt(instruct_tune_dataset["test"][2])
print(prompt)

[INST] <<SYS>>
Put yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges.
Here is my question: Vertebrata are characterized by the presence of what?[\INST]
Expert Response: backbone


### **Initializing the Model**
* Load the model using a 4-bit configuration, employing double quantization, and set bfloat16 as the compute data type.

* Notably, we opt for the instruct-tuned model in this instance rather than the base model. It's worth mentioning that fine-tuning a base model necessitates a more substantial amount of data!

In [21]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)


bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [22]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

* **Model and Tokenizer:** https://huggingface.co/NousResearch/Llama-2-7b-chat-hf

In [23]:
mode_id = "NousResearch/Llama-2-7b-chat-hf"

In [24]:
model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto", use_cache=False
    )

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [25]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

### **Let's example how well the model does at this task currently  (without training):**
* `do_sample:`This parameter controls whether to use sampling during text generation. When do_sample is set to True, the model samples words for the next token based on their probabilities. If set to False, the model uses greedy decoding and selects the token with the highest probability at each step.
* `pad_token_id:` This parameter represents the token ID used for padding in the tokenizer. In text generation, it's common to set the padding token to the end-of-sequence token (eos_token_id) to ensure that the generated text is complete.
* `top_k:` top_k controls the number of highest probability options considered during sampling. For each token, the model will limit the sampling to the top k tokens with the highest probabilities.
* **High top_k Value:** More deterministic and focused generation. The
model considers a larger number of top probability options, resulting in conservative and predictable outputs.
* **Low top_k Value:** More randomness in sampling. The model restricts the number of top probability options, leading to diverse and creative text generation, but it may result in less coherent or focused output.

* `top_p:` top_p is used in nucleus sampling, where the model considers the cumulative probability mass of the top tokens and stops sampling when it reaches a predefined threshold. It helps in preventing the model from being overly verbose and encourages diversity in the generated text.
* **High top_p Value:** Promotes diverse and varied text generation by considering a larger range of probable tokens. It's good for creating exploratory and varied outputs.
* **Low top_p Value:** Adopts a stricter threshold for selecting tokens, resulting in more conservative and focused text generation. This may lead to more deterministic outputs closely tied to the most probable tokens.
* `temperature:` temperature controls the level of randomness in the sampling process. A higher temperature (e.g., 1.0) increases randomness, making the generated text more diverse. Lower values (e.g., 0.8) make the sampling more deterministic, with the model focusing on higher probability tokens.

**Documentation Link:**
* https://python.langchain.com/docs/integrations/llms/huggingface_pipelines


In [26]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline
from langchain.prompts import PromptTemplate

# Additional configuration parameters
do_sample = True
pad_token_id = tokenizer.eos_token_id
top_p = 0.8  # You can adjust this value based on your preference
temperature = 0.8  # You can adjust this value based on your preference

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256, do_sample=do_sample, pad_token_id=pad_token_id,  top_p=top_p, temperature=temperature)
llm = HuggingFacePipeline(pipeline=pipe)


# Creating prompt for large language model
pre_prompt = """[INST] <<SYS>>\nPut yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges.\n"""
prompt = pre_prompt + "Here is my question: {question}" +"[\INST]"

prompt = PromptTemplate(template=prompt, input_variables=["question"])


chain = prompt | llm

question = "The lens focuses light on the retina , which covers the back of the inside of the eye. the retina has light-sensing photoreceptor cells called?"

print(chain.invoke({"question": question}))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


  Great, I'm here to help! The photoreceptor cells in the retina that are responsible for sensing light are called rods and cones.

Rods are sensitive to low light levels and are responsible for night vision and peripheral vision. They are also more sensitive to movement and can detect changes in light intensity more quickly than cones.

Cones, on the other hand, are sensitive to brighter light levels and are responsible for color vision and fine detail vision. There are three types of cones, each sensitive to different wavelengths of light, which allows us to see a wide range of colors.

Both rods and cones are connected to the brain through the optic nerve, which carries visual information from the eye to the brain for processing.

I hope that helps! Do you have any other questions about the eye or vision?


### **Setting up the Training**
we will be using the `huggingface` and the `peft` library!

In [27]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")


* we need to prepare the model to be trained in 4bit so we will use the  **`prepare_model_for_kbit_training`** function from peft




In [28]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# **Training Hyperparameters**
The choice of hyperparameters is contingent upon the desired training duration. Pay special attention to the following key factors:

* `num_train_epochs/max_steps:` Dictates the number of iterations over the data. Exercise caution, as an excessive number may lead to overfitting!

* `learning_rate:` Governs the convergence speed of the model. Adjust this parameter judiciously for optimal results.

In [29]:
from transformers import TrainingArguments
output_model= "Llama_instruct_generation"
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=1,
        max_steps=100,
        fp16=True,
)


### **Setting up the trainer**

`max_seq_length`: Context window size


In [30]:
from trl import SFTTrainer

max_seq_length = 256

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt,  # This will apply the create_prompt mapping to all training and test dataset
  args=training_arguments,
  train_dataset=instruct_tune_dataset['train'],
  eval_dataset=instruct_tune_dataset['test']
)


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:310: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


### **Training starts here**

In [31]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.209200
20,1.490800
30,1.018300
40,0.753100
50,0.632000
60,0.607000
70,0.595500
80,0.543600
90,0.565100
100,0.584400


TrainOutput(global_step=100, training_loss=0.8998926258087159, metrics={'train_runtime': 513.502, 'train_samples_per_second': 0.779, 'train_steps_per_second': 0.195, 'total_flos': 4062128898048000.0, 'train_loss': 0.8998926258087159, 'epoch': 0.32})

### **Save the model**

In [32]:
trainer.save_model("Llama_instruct_generation")

In [33]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:249: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


#### **Generate Answers After Training**

* `invoke:` call the chain on an input
* `batch:` call the chain on a list of inputs
#### **Documentation Link:**
* https://python.langchain.com/docs/expression_language/interface

# **Conversation Buffer memory**

In [34]:
# Creating prompt for large language model
pre_prompt = """[INST] <<SYS>>\nPut yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges.\n"""
prompt = pre_prompt + "Here is my question: {question}" +"[\INST]"

prompt = PromptTemplate(template=prompt, input_variables=["question"])

In [35]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain

pipeline = pipeline("text-generation", model=merged_model, tokenizer=tokenizer, max_new_tokens=256, do_sample=do_sample, pad_token_id=pad_token_id, top_p=top_p, temperature=temperature)
llm = HuggingFacePipeline(pipeline=pipeline)

memory = ConversationBufferMemory()
chat_llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

#### **Example: 01**

In [37]:
question= "What are people with osteoporosis at increased risk of?"
chat_llm_chain.predict(question=question)



> Entering new LLMChain chain...
Prompt after formatting:
[INST] <<SYS>>
Put yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges.
Here is my question: What are people with osteoporosis at increased risk of?[\INST]


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



> Finished chain.


"  Great, I'm here to help! People with osteoporosis are at increased risk of fractures, particularly of the spine, hips, and wrists. This is because osteoporosis is a condition where the bones become weak and brittle, making them more susceptible to fractures. In fact, osteoporosis is the most common cause of fractures in older adults.\n\nThere are several reasons why people with osteoporosis are at higher risk of fractures. One reason is that the bones in people with osteoporosis are less dense and more prone to breaking. Additionally, the bones may lose their structural integrity, which can make them more susceptible to fractures.\n\nSome common fractures that people with osteoporosis may experience include:\n\n* Vertebral fractures: These are fractures of the spine, which can cause back pain, loss of height, and difficulty standing or walking.\n* Hip fractures: These are fractures of the hip bone, which can cause pain, difficulty walking, and increased risk of mort"

#### **Example: 02**

In [38]:
question = "The body cannot sustain for very long the bursts of energy mediated by epinephrine and norepinephrine, hormones associated with what stress response?"
chat_llm_chain.predict(question=question)



> Entering new LLMChain chain...
Prompt after formatting:
[INST] <<SYS>>
Put yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges.
Here is my question: The body cannot sustain for very long the bursts of energy mediated by epinephrine and norepinephrine, hormones associated with what stress response?[\INST]


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



> Finished chain.


"  Great, I'm here to help! The body's stress response is mediated by hormones such as epinephrine and norepinephrine, which are produced by the adrenal glands in response to stressful situations. These hormones help the body to mobilize energy and prepare for action by increasing heart rate, blood pressure, and metabolism. However, the body cannot sustain these high levels of energy for very long, and prolonged exposure to these hormones can lead to negative consequences such as fatigue, anxiety, and cardiovascular disease. So, the question you're asking is an important one, and I'm happy to help you understand it better! Can you tell me more about what you're looking for in terms of information or insights?"

#### **Example: 03**

In [39]:
question = "What system enters the brain stem and influences activity in the cerebellum, spinal cord, and cerebral cortex?"
chat_llm_chain.predict(question=question)



> Entering new LLMChain chain...
Prompt after formatting:
[INST] <<SYS>>
Put yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges.
Here is my question: What system enters the brain stem and influences activity in the cerebellum, spinal cord, and cerebral cortex?[\INST]


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



> Finished chain.


'  Great, I\'m here to help you with your question! The system that enters the brain stem and influences activity in the cerebellum, spinal cord, and cerebral cortex is called the autonomic nervous system (ANS).\n\nThe ANS is a part of the nervous system that controls involuntary actions, such as heart rate, blood pressure, digestion, and breathing. It is composed of two subsystems: the sympathetic nervous system (SNS) and the parasympathetic nervous system (PNS).\n\nThe SNS is responsible for the "fight or flight" response, which prepares the body for action by increasing heart rate, blood pressure, and respiration. It is activated in response to stress, excitement, or danger, and helps the body to mobilize energy and resources to respond to the situation.\n\nThe PNS, on the other hand, is responsible for the "rest and digest" response, which helps the body to relax and recover after the "fight or flight" response. It is activated when the body is in a state of relaxation or when it n

In [40]:
memory.load_memory_variables({})

{'history': 'Human: The lens focuses light on the retina , which covers the back of the inside of the eye. the retina has light-sensing photoreceptor cells called?\nAI:   Great, I\'m here to help! The light-sensing photoreceptor cells in the retina are called rods and cones.\nRods are sensitive to low light levels and are responsible for peripheral and night vision. They are also more sensitive to movement and are important for detecting changes in the visual environment.\nCones, on the other hand, are sensitive to higher light levels and are responsible for color vision and fine detail vision. There are three types of cones, each sensitive to different wavelengths of light, which allows us to see a wide range of colors.\nSo, to summarize, the retina contains rods and cones, which are light-sensing photoreceptor cells that play a crucial role in our ability to see and interpret visual information.\nHuman: What are people with osteoporosis at increased risk of?\nAI:   Great, I\'m here t

In [41]:
print(memory.buffer)

Human: The lens focuses light on the retina , which covers the back of the inside of the eye. the retina has light-sensing photoreceptor cells called?
AI:   Great, I'm here to help! The light-sensing photoreceptor cells in the retina are called rods and cones.
Rods are sensitive to low light levels and are responsible for peripheral and night vision. They are also more sensitive to movement and are important for detecting changes in the visual environment.
Cones, on the other hand, are sensitive to higher light levels and are responsible for color vision and fine detail vision. There are three types of cones, each sensitive to different wavelengths of light, which allows us to see a wide range of colors.
So, to summarize, the retina contains rods and cones, which are light-sensing photoreceptor cells that play a crucial role in our ability to see and interpret visual information.
Human: What are people with osteoporosis at increased risk of?
AI:   Great, I'm here to help! People with o

# **Conversation Buffer Window Memory**
* ConversationBufferWindowMemory keeps a list of the interactions of the conversation over time. It only uses the last K interactions. This can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

In [42]:
# Creating prompt for large language model
pre_prompt = """[INST] <<SYS>>\nPut yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges.\n"""
prompt = pre_prompt + "Here is my question: {question}" +"[\INST]"

prompt = PromptTemplate(template=prompt, input_variables=["question"])

In [43]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import LLMChain

pipeline = pipeline("text-generation", model=merged_model, tokenizer=tokenizer, max_new_tokens=256, do_sample=do_sample, pad_token_id=pad_token_id, top_p=top_p, temperature=temperature)
llm = HuggingFacePipeline(pipeline=pipeline)

memory = ConversationBufferWindowMemory(k=2)
chat_llm_chain_window = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

#### **Example: 01**

In [44]:
question = "Although magma once filled our moon's craters, what is thought to have ended there over a billion years ago?"
chat_llm_chain_window.predict(question=question)



> Entering new LLMChain chain...
Prompt after formatting:
[INST] <<SYS>>
Put yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges.
Here is my question: Although magma once filled our moon's craters, what is thought to have ended there over a billion years ago?[\INST]


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



> Finished chain.


"  Ah, an excellent question! 🤔 The moon's craters were once filled with magma, but what is thought to have ended there over a billion years ago is the lunar magma ocean. 🌕\n\nThe lunar magma ocean was a vast, molten sea of magma that covered the entire surface of the moon. It is believed to have formed early in the moon's history, during a period known as the lunar magma ocean phase, which occurred between 4.5 and 3.8 billion years ago. 🕰️ During this time, the moon was still in its formative stages, and the intense heat generated by the moon's early collisions caused the rocky material to melt and form a magma ocean. 💦\n\nHowever, over time, the lunar magma ocean is thought to have cooled and solidified, eventually forming the craters we see on the moon's surface today. This process is believed to have occurred over a billion years ago, and it marked the end of the lunar magma ocean phase. 🌊\n\nSo, to summarize, what is"

#### **Example: 02**

In [45]:
question = "Humans are among the most versatile of mammals with what type of diet?"
chat_llm_chain_window.predict(question=question)



> Entering new LLMChain chain...
Prompt after formatting:
[INST] <<SYS>>
Put yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges.
Here is my question: Humans are among the most versatile of mammals with what type of diet?[\INST]


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



> Finished chain.


'  Great question! Humans are indeed one of the most versatile mammals, and their diet can vary greatly depending on their geographical location, cultural background, and individual preferences.\n\nHumans are omnivores, which means they can eat and digest both plants and animals. Our diet has evolved over time and has been influenced by a variety of factors, including geography, climate, and cultural traditions.\n\nIn general, humans have a diverse diet that includes a wide range of foods, such as:\n\n* Grains: bread, rice, pasta, and other starchy foods\n* Vegetables: leafy greens, fruits, and other plant-based foods\n* Fruits: apples, bananas, berries, and other sweet or tart fruits\n* Proteins: meat, poultry, fish, eggs, and dairy products\n* Fats and oils: olive oil, butter, and other types of oil\n* Sweets: candy, chocolate, and other sugary treats\n\nOf course, this is just a general overview of the human diet, and there is'

#### **Example: 03**

In [46]:
question = "Nephrons, renal tubules and the loop of henle are part of the process of blood filtration by what organs?"
chat_llm_chain_window.predict(question=question)



> Entering new LLMChain chain...
Prompt after formatting:
[INST] <<SYS>>
Put yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges.
Here is my question: Nephrons, renal tubules and the loop of henle are part of the process of blood filtration by what organs?[\INST]


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



> Finished chain.


'  Great question! The organs responsible for blood filtration are the kidneys. Nephrons, renal tubules, and the loop of Henle are all components of the kidneys that play a crucial role in the process of blood filtration.\n\nNephrons are the functional units of the kidneys that are responsible for filtering blood. Each nephron is made up of a glomerulus (a tiny ball of blood vessels) and a tubule (a narrow tube-like structure). The glomerulus acts as a filter, allowing small molecules like water, ions, and waste products to pass through while keeping larger molecules like proteins and red blood cells behind. The tubule then further filters the blood, removing any remaining waste products and excess fluids.\n\nRenal tubules are the tubes that line the nephrons and play a crucial role in the reabsorption of water and electrolytes. These tubules have specialized cells that can selectively reabsorb water and electrolytes back into the bloodstream, helping to maintain proper fluid balance i

In [47]:
memory.load_memory_variables({})

{'history': 'Human: Humans are among the most versatile of mammals with what type of diet?\nAI:   Great question! Humans are indeed one of the most versatile mammals, and their diet can vary greatly depending on their geographical location, cultural background, and individual preferences.\n\nHumans are omnivores, which means they can eat and digest both plants and animals. Our diet has evolved over time and has been influenced by a variety of factors, including geography, climate, and cultural traditions.\n\nIn general, humans have a diverse diet that includes a wide range of foods, such as:\n\n* Grains: bread, rice, pasta, and other starchy foods\n* Vegetables: leafy greens, fruits, and other plant-based foods\n* Fruits: apples, bananas, berries, and other sweet or tart fruits\n* Proteins: meat, poultry, fish, eggs, and dairy products\n* Fats and oils: olive oil, butter, and other types of oil\n* Sweets: candy, chocolate, and other sugary treats\n\nOf course, this is just a general ov

In [48]:
print(memory.buffer)

Human: Humans are among the most versatile of mammals with what type of diet?
AI:   Great question! Humans are indeed one of the most versatile mammals, and their diet can vary greatly depending on their geographical location, cultural background, and individual preferences.

Humans are omnivores, which means they can eat and digest both plants and animals. Our diet has evolved over time and has been influenced by a variety of factors, including geography, climate, and cultural traditions.

In general, humans have a diverse diet that includes a wide range of foods, such as:

* Grains: bread, rice, pasta, and other starchy foods
* Vegetables: leafy greens, fruits, and other plant-based foods
* Fruits: apples, bananas, berries, and other sweet or tart fruits
* Proteins: meat, poultry, fish, eggs, and dairy products
* Fats and oils: olive oil, butter, and other types of oil
* Sweets: candy, chocolate, and other sugary treats

Of course, this is just a general overview of the human diet, a